In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import pickle
import utils
import moran_auto
import gmm
import kmeans_utils
import figure_functions
from sklearn.preprocessing import MinMaxScaler
from sklearn import mixture
from sklearn.cluster import KMeans
import csv
from collections import defaultdict

In [3]:
curr_dir = os.getcwd()
data_path = curr_dir + '/../data/'
fig_path = curr_dir + '/../figs/'
results_path = curr_dir + '/../results'

# Train and Test GMM

In [4]:
park_data, gps_loc, N = utils.load_daily_data(data_path)

In [96]:
check = np.array(sorted(park_data['Load'].values.tolist(), reverse=True))

In [106]:
len(np.where(check > 2)[0])/float(len(check)) * 100

0.33889494509043927

In [5]:
results = gmm.locational_demand_analysis(park_data, gps_loc, N)

In [15]:
days = [result[0] for result in results]
hours = [result[1] for result in results]
errors = [result[2] for result in results]
morans_mix = [result[3] for result in results]
morans_adj = [result[4] for result in results]
means = [result[5] for result in results]

In [44]:
scores, times = kmeans_utils.get_time_scores(means)

In [61]:
best_times = {}
best_days = {}
for time in times:
    day = time/12
    hour = time%12 + 8
    if hour not in best_times:
        best_times[hour] = time
    if day not in best_days and hour not in [8, 15,16,17]:
        print day, hour
        best_days[day] = time

0 9
4 13
5 9
1 14
3 19
2 18


In [ ]:
good_times = []

for time in times:
    if time % 12 + 8 in [8,9]:
        continue
        
    good_times.append(time)
    data = np.vstack((means[time]))
    kmeans = KMeans(n_clusters=3).fit(data)
    labels = kmeans.labels_.tolist()

In [62]:
good_times = best_days.values()

In [63]:
fig, ax = figure_functions.centroid_plots(means, gps_loc, N, times=good_times, fig_path=fig_path, shape=(2,3))